# Testing if an RL-Agent can beat Protostar
## Approach:
- Train a sequence model in a supervised approach to create strings which result in a buffer overflow given the assembly code
- See if it generalizes to differenet lengths and slight variations in the assemembly
- Let a RL-Agent learning via trial and error and see if it is better than a brute force strategy

## Note:
- for now this will be a quick and dirty implementation

## Sourcecode of stack0 which will be used for pre-training

<pre>
    0x080483f4 < main+0 >:    push   %ebp
    0x080483f5 < main+1 >:    mov    %esp,%ebp
    0x080483f7 < main+3 >:    and    $0xfffffff0,%esp
    0x080483fa < main+6 >:    sub    $0x60,%esp
    0x080483fd < main+9 >:    movl   $0x0,0x5c(%esp)
    0x08048405 < main+17 >:   lea    0x1c(%esp),%eax
    0x08048409 < main+21 >:   mov    %eax,(%esp)
    0x0804840c < main+24 >:   call   0x804830c < gets@plt >
    0x08048411 < main+29 >:   mov    0x5c(%esp),%eax
    0x08048415 < main+33 >:   test   %eax,%eax
    0x08048417 < main+35 >:   je     0x8048427 < main+51 >
    0x08048419 < main+37 >:   movl   $0x8048500,(%esp)
    0x08048420 < main+44 >:   call   0x804832c < puts@plt >
    0x08048425 < main+49 >:   jmp    0x8048433 < main+63 >
    0x08048427 < main+51 >:   movl   $0x8048529,(%esp)
    0x0804842e < main+58 >:   call   0x804832c < puts@plt >
    0x08048433 < main+63 >:   leave  
    0x08048434 < main+64 >:   ret

## Preprocessing to get the sourcecode to a numeric representation

In [1]:
import numpy as np
data = open('assembly_code/assembly1.txt', 'r').read()

In [2]:
def remove_line_nr_main(data):
    while(True):
        start_index = data.find('<main+')
        end_index = start_index + data[start_index:].find('>') + 1
        data = data[0:start_index] + data[end_index:]
        if(start_index == -1):
            break
    return data

data = remove_line_nr_main(data)

In [3]:
data1 = data.replace(' :', '') 
data2 = data1.replace('\t', ' ') 
data3 = data2.replace('$', '') 
data4 = data3.split('\n')

# if the last char is a space, delete it
i=0
for row in data4:
    try:
        if row[-1] == ' ':
            data4[i] = row[:-1]
    except:
        print('last row')
    i += 1
        

new_data = []
data4
for row in data4:
    addr_lineNr_cmd_spaces_args = row.split(' ')
    try:
        
        new_data.append([addr_lineNr_cmd_spaces_args[0], addr_lineNr_cmd_spaces_args[1], addr_lineNr_cmd_spaces_args[-1]])
    except:
        print('last_line')


last row
last_line


<pre>
    every address becomes an integer number
    every assembly command is a 1 in a one hot representation
    the 1 or 2 arguments are 1-hot for registers and function calls or plain numbers otherwise
    integer, fixed length one hot, [fixed length one hot, integer, fixed legth one hot, integer]

In [4]:
assembly_instructions = []
for row in new_data:
    assembly_instructions.append(row[1])
assembly_instructions_enc = set(assembly_instructions)
instr_to_ix = { instr:i for i,instr in enumerate(sorted(assembly_instructions_enc)) }
ix_to_instr = { i:instr for i,instr in enumerate(sorted(assembly_instructions_enc)) }

instr_to_ix

{'and': 0,
 'call': 1,
 'je': 2,
 'jmp': 3,
 'lea': 4,
 'leave': 5,
 'mov': 6,
 'movl': 7,
 'push': 8,
 'ret': 9,
 'sub': 10,
 'test': 11}

In [5]:
fct_args = []
for row in new_data:
    args = row[2:] # everything after the assembly command for each line
    args = args[0].split(',') # function arguments are seperated by comma
    for arg in args:
        if(arg[0:2] == '0x'):
            if arg.find('(') == -1: # keine klammer
                # we got a number
                print(arg)

            else:
                # 0x3c(esp)
                arg = arg.replace(')', '')
                arg = arg.split('(')
                for subarg in arg:
                    if(subarg[0:2] != '0x'):
                        fct_args.append(subarg)
        else:
            # (esp)
            #<plt<main+
            arg = arg.replace(')', '')
            arg = arg.replace('(', '')
            fct_args.append(arg)

            
fct_args.append('0xfffffff0')


0xfffffff0
0x60
0x0
0x8048427
0x8048500
0x8048433
0x8048529


In [6]:
fct_args_enc = set(fct_args)
fct_args_to_ix = { arg:i for i,arg in enumerate(sorted(fct_args_enc)) }
ix_to_fct_args = { i:arg for i,arg in enumerate(sorted(fct_args_enc)) }

In [7]:
code_list = []
for row in new_data:
    for word in row:
        code_list.append(word)

In [8]:
code_words = set(code_list)
words_to_ix = { words:i for i,words in enumerate(sorted(code_words)) }
ix_to_word = { i:words for i,words in enumerate(sorted(code_words)) }

In [9]:
prep_as = np.zeros([18, 6])
i = 0 # instruction counter
for row in new_data:
    # encode memory segment
    prep_as[i, 0] = int(row[0], 16)
    
    # encode assembly command
    prep_as[i, 1] = instr_to_ix[row[1]]
    
    # encode function arguments
    args = row[2].split(',')
    
    j = 0 # argument counter
    for arg in args:
        if(arg[0:2] == '0x'):
            if arg.find('(') == -1: # in case of a hex value
                if arg == '0xfffffff0':
                    prep_as[i, 2+j] = fct_args_to_ix[arg]
                else: 
                    prep_as[i, 2+j] = (int(arg, 16) )
            else:
                # 0x3c(esp)
                arg = arg.replace(')', '')
                arg = arg.split('(')
                prep_as[i, 2+j] = int(arg[0], 16)
                prep_as[i, 3+j] = fct_args_to_ix[arg[1]]
        else:
            # (esp)
            #<plt<main+
            arg = arg.replace(')', '')
            arg = arg.replace('(', '')
            prep_as[i, 3+j] = fct_args_to_ix[arg]
        j += 2
    i+=1

In [10]:
for i in range(prep_as.shape[0]):
    for j in range(2, 6):
        if prep_as[i, j] > prep_as[0, 0]:
            print(prep_as[i, j] - prep_as[0, 0])
            prep_as[i, j] = prep_as[i, j] - prep_as[0, 0]
            
prep_as[:,0] = prep_as[:,0]-prep_as[0,0]

51.0
268.0
63.0
309.0


# generate data

we need a similar amount of samples for each class otherwise rare classes get ignored

In [11]:
sample_size = 10000
x = np.zeros([sample_size, prep_as.shape[0], prep_as.shape[1]])
y = np.zeros([sample_size, 150, 3])

for i in range(sample_size):
    x[i,:,:] = prep_as
    #buffer_size = np.random.randint(10, 100)
    buffer_size = np.random.randint(40, 80)
    x[i,3,2] = buffer_size
    y[i, :buffer_size, 0] = 1
    y[i, buffer_size:buffer_size+50, 1] = 1
    y[i, buffer_size+50:, 2] = 1

x_val = x[-2000:, :, :]
y_val = y[-2000:, :, :]

x_train = x[:-2000:, :, :]
y_train = y[:-2000:, :, :]

# neural net for prediction

In [17]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import SGD

RNN = layers.GRU
HIDDEN_SIZE = 4
BATCH_SIZE = 8
LAYERS = 1

print('Build model...')
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(x.shape[1], x.shape[2])))

model.add(layers.RepeatVector( 150 ))
model.add(RNN(4, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
#sgd = SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False, clipnorm=1., clipvalue=0.5)

model.add(layers.TimeDistributed(layers.Dense(3, activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

# Train the model each generation and show predictions against the validation
# dataset.

for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
        

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_11 (GRU)                 (None, 4)                 132       
_________________________________________________________________
repeat_vector_6 (RepeatVecto (None, 150, 4)            0         
_________________________________________________________________
gru_12 (GRU)                 (None, 150, 4)            108       
_________________________________________________________________
time_distributed_6 (TimeDist (None, 150, 3)            15        
Total params: 255
Trainable params: 255
Non-trainable params: 0
_________________________________________________________________

--------------------------------------------------
Iteration 1
Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 103s 13ms/step - loss: 0.7435 - acc: 0.6634 - val_loss: 0.5478 - val_acc: 0.7836

------------

KeyboardInterrupt: 

## testing if the algorithm also learns from raw input 

In [22]:
from keras.models import Sequential
from keras import layers
import numpy as np

data = open('assembly_code/assembly1.txt', 'r').read()

class StringEncoder(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
        
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)
    

MAXLEN = 800
chars = '0123456789abcdefghijklmnopqrstuvwxyz<>+@%$:\t\n,() '
stringEncoder = StringEncoder(chars)
questions = []
expected = []
seen = set()
print('Generating data...')
for i in range(20,80):
    data2 = data
    randn = hex(i)
    query = data2.replace('0x5c', randn)
    ans = i * 'a' + 'bbb' + ' ' * (103 - 3 - i)
    
    questions.append(query)
    expected.append(ans)
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), 103, len(chars)), dtype=np.bool)

for i, sentence in enumerate(questions):
    x[i] = stringEncoder.encode(sentence, MAXLEN)

for i, sentence in enumerate(expected):
    y[i] = stringEncoder.encode(sentence, 103)

indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Generating data...
Vectorization...
Training Data:
(54, 800, 49)
(54, 103, 49)
Validation Data:
(6, 800, 49)
(6, 103, 49)


In [23]:
RNN = layers.LSTM
HIDDEN_SIZE = 400
BATCH_SIZE = 32
LAYERS = 3
print('Build model...')
model = Sequential()

model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(103))

for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=20,
              validation_data=(x_val, y_val))

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 400)               720000    
_________________________________________________________________
repeat_vector_7 (RepeatVecto (None, 103, 400)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 103, 400)          1281600   
_________________________________________________________________
lstm_3 (LSTM)                (None, 103, 400)          1281600   
_________________________________________________________________
lstm_4 (LSTM)                (None, 103, 400)          1281600   
_________________________________________________________________
time_distributed_7 (TimeDist (None, 103, 49)           19649     
Total params: 4,584,449
Trainable params: 4,584,449
Non-trainable params: 0
___________________________________________________

54/54 [==============================] - 2s 31ms/step - loss: 0.7512 - acc: 0.4833 - val_loss: 0.7578 - val_acc: 0.4563
Epoch 13/20
54/54 [==============================] - 2s 31ms/step - loss: 0.8204 - acc: 0.4833 - val_loss: 0.8008 - val_acc: 0.4563
Epoch 14/20
54/54 [==============================] - 2s 30ms/step - loss: 0.7742 - acc: 0.6215 - val_loss: 0.7599 - val_acc: 0.5922
Epoch 15/20
54/54 [==============================] - 2s 31ms/step - loss: 0.7837 - acc: 0.5574 - val_loss: 0.7819 - val_acc: 0.5728
Epoch 16/20
54/54 [==============================] - 2s 30ms/step - loss: 0.7982 - acc: 0.5498 - val_loss: 0.7506 - val_acc: 0.6019
Epoch 17/20
54/54 [==============================] - 2s 31ms/step - loss: 0.7765 - acc: 0.5394 - val_loss: 0.7808 - val_acc: 0.4563
Epoch 18/20
54/54 [==============================] - 2s 30ms/step - loss: 0.7765 - acc: 0.4833 - val_loss: 0.7681 - val_acc: 0.4563
Epoch 19/20
54/54 [==============================] - 2s 31ms/step - loss: 0.7637 - acc: 

Epoch 12/20
54/54 [==============================] - 2s 31ms/step - loss: 0.6917 - acc: 0.6516 - val_loss: 0.7119 - val_acc: 0.6311
Epoch 13/20
54/54 [==============================] - 2s 31ms/step - loss: 0.7446 - acc: 0.5962 - val_loss: 0.6820 - val_acc: 0.6214
Epoch 14/20
54/54 [==============================] - 2s 31ms/step - loss: 0.7442 - acc: 0.5307 - val_loss: 0.7860 - val_acc: 0.4563
Epoch 15/20
54/54 [==============================] - 2s 31ms/step - loss: 0.7781 - acc: 0.4833 - val_loss: 0.7637 - val_acc: 0.4563
Epoch 16/20
54/54 [==============================] - 2s 31ms/step - loss: 0.7611 - acc: 0.4833 - val_loss: 0.7429 - val_acc: 0.8803
Epoch 17/20
54/54 [==============================] - 2s 32ms/step - loss: 0.7505 - acc: 0.8261 - val_loss: 0.7484 - val_acc: 0.6214
Epoch 18/20
54/54 [==============================] - 2s 31ms/step - loss: 0.7531 - acc: 0.5944 - val_loss: 0.7243 - val_acc: 0.6214
Epoch 19/20
54/54 [==============================] - 2s 31ms/step - loss: 0.

Epoch 12/20
54/54 [==============================] - 2s 32ms/step - loss: 0.4495 - acc: 0.8278 - val_loss: 0.4024 - val_acc: 0.8738
Epoch 13/20
54/54 [==============================] - 2s 31ms/step - loss: 0.4455 - acc: 0.8234 - val_loss: 0.4076 - val_acc: 0.8463
Epoch 14/20
54/54 [==============================] - 2s 31ms/step - loss: 0.5791 - acc: 0.7573 - val_loss: 0.5355 - val_acc: 0.7751
Epoch 15/20
54/54 [==============================] - 2s 31ms/step - loss: 0.6787 - acc: 0.7051 - val_loss: 0.6707 - val_acc: 0.6602
Epoch 16/20
54/54 [==============================] - 2s 31ms/step - loss: 0.7033 - acc: 0.6411 - val_loss: 0.6382 - val_acc: 0.7379
Epoch 17/20
54/54 [==============================] - 2s 31ms/step - loss: 0.6385 - acc: 0.7769 - val_loss: 0.8955 - val_acc: 0.4563
Epoch 18/20
54/54 [==============================] - 2s 31ms/step - loss: 0.8505 - acc: 0.4833 - val_loss: 0.7998 - val_acc: 0.4563
Epoch 19/20
54/54 [==============================] - 2s 31ms/step - loss: 0.

KeyboardInterrupt: 